<a href="https://colab.research.google.com/github/seunghee0518/R_File/blob/main/R_%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95_6%EC%9E%A5_%EB%AA%A8%ED%98%95%EC%9D%98%EC%A7%84%EB%8B%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 오차의 등분산성 검정 : 스코어검정
* <표 6.1> 흰기러기 데이터
* photo(Y) : 사진 
* obsA(X1) : 관측자A
* obsB(X2) : 관측자B

In [ ]:
#데이터 가져오기
goose_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95/goose.txt'
goose = read.table(goose_url, header = T)
head(goose, 3)

In [ ]:
#회귀모형 적합 : obsA
goose.lm = lm(photo ~ obsA, data = goose)
plot(goose.lm$fitted, goose.lm$resid, pch =19)
text(goose.lm$fitted, goose.lm$resid, labels = goose$photo, pos = 1)

In [ ]:
#스코어 검정 : car패키지 - ncvTest()
install.packages('car')
library(car)
ncvTest(goose.lm)

* 스코어 검정의 통계량값 : Chisquare = 81.41318
* 자유도가 1인 X2(1)값과 비교해 검정함
* 유의확률 p값 : 1.83e-19 -> 0.05보다 매우 작으므로 등분산가정(귀무가설)을 기각함
* 즉, 설명변수X1이 분산의 크기에 영향을 미쳐 등분산가정을 만족하지 않음

# 회귀모형의 비선형성

In [ ]:
#데이터
tree_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95/tree.txt'
tree = read.table(tree_url, header = T)
head(tree, 3)

In [ ]:
#회귀 적합
tree.lm = lm(V ~ D+H, data = tree)

In [ ]:
#잔차산점도
plot(tree$D, tree.lm$resid, pch = 19)

* 변수D와 잔차의 산점도에서 2차함수 형태의 비선형모양이 뚜렷하게 나타남
* 즉. 모형의 선형성 가정에 위배됨을 알 수 있음
* 이 경우 변수D^2항을 모형에 포함시켜도 좋을 것 같음 

In [ ]:
plot(tree$H, tree.lm$resid, pch = 19)

* 어떤 뚜렷한 현상은 보이지 않음

# 오차의 정규성
* <표 6.1> 흰기러기 데이터
* 잔차산점도와 스코어 검정을 통해 등분산 가정에 문제가 있다는 것을 이미 확인함
* 정규확률그림을 통해 오차정규성 여부 확인

In [ ]:
#흰기러기 데이터
goose_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95/goose.txt'
goose = read.table(goose_url, header = T)
head(goose, 3)

In [ ]:
#car 패키지 - qqPlot()
install.packages('car')
library(car)
goose.lm = lm(photo ~ obsA, data = goose)
qqPlot(goose.lm)

* 잔차가 직선의 형태를 따름 : 정규성 가정에 문제가 없다
* 해당 그래프는 잔차가 곡선의 형태이므로 정규성 가정에 위배된다고 판단됨

In [ ]:
# W통계량으로 정규성 검정 : mvnormtest패키지-shapiro.test()
install.packages('mvnormtest')
library(mvnormtest)
goose.rstudent = rstudent(goose.lm)
shapiro.test(goose.rstudent)

* W통계량 : 0.7192
* p값 : 5.97e-8
* W통계량과 p값이 매우 작으므로 스튜던트화 잔차가 정규분포를 따르지 않음
* 따라서 해당 자료는 등분산성을 만족하지 않고 오차의 정규성을 따르지 않음

# Box-Cox의 변환 : 변수의 변환
* X : 월 총에너지 사용량
* Y : 시간당 최대수요


In [ ]:
#데이터 가져오기
energy_url = 'https://raw.githubusercontent.com/seunghee0518/R_File/main/%ED%9A%8C%EA%B7%80%EB%AA%A8%ED%98%95/energy.txt'
energy = read.table(energy_url, header = T)
head(energy, 3)

In [ ]:
#잔차산점도
energy.lm = lm(Y ~ X, data = energy)
plot(energy.lm$fitted, energy$resid, pch = 19)

* 잔차산점도의 형태로 보아 등분산가정에 의심됨

In [ ]:
#경험적으로 변수변환 : Y -> 루트로 변수변환
energy$sqrtY = sqrt(energy$Y)
energy.lm2 = lm(sqrtY ~ X, data = energy)
plot(energy.lm2$fitted, energy.lm2$resid, pch = 19)

In [ ]:
#Box-Cox변환방법 : MASS패키지 - boxcox()
install.packages('MASS')
library(MASS)
boxcox(Y ~ X, data = energy, lambda = seq(-2, 2, 1/2), plotit = TRUE)

* log-likelihood 값이 최대일때를 찾으면 됨
* 즉, 0.5일때 최대임
* 따라서 이는 루트 변수변환에 해당함을 알 수 있음